In [1]:
from TinyImageNetLoader import TinyImageNetDataset
import torchvision.transforms as transforms
import torch

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

valset = TinyImageNetDataset("/datasets/tiny-imagenet-200", mode="val", transform=val_transform)
#print(next(enumerate(validation_loader)))

train_transform =  transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(0.5),
    normalize
])

trainset = TinyImageNetDataset("/datasets/tiny-imagenet-200", transform=train_transform)

Preloading train data...: 100%|██████████| 100000/100000 [11:47<00:00, 141.37it/s]


In [2]:
validation_loader = torch.utils.data.DataLoader(
        valset, batch_size=256, shuffle=False, num_workers=4)
training_loader = torch.utils.data.DataLoader(
        trainset, batch_size=256, shuffle=True, num_workers=4)

In [7]:
import torch.nn as nn
eps=1e-10
def ratio_loss_fn(out, labels, predicate_matrix):
    out = out.view(-1, 1, NUM_FEATURES) # out is a batch of 1D binary vectors
    ANDed = out * predicate_matrix # AND operation
    diff = ANDed - out # Difference of ANDed and out => if equal, then out is a subset of its class' predicates

    entr_loss = torch.nn.CrossEntropyLoss()
    loss_cl = entr_loss(diff.sum(dim=2), labels) # Is "out" a subset of its class' predicates?

    batch_size = out.shape[0]

    out = out.view(-1, NUM_FEATURES)
    diff_square = (out - predicate_matrix[labels]).pow(2)
    
    false_positives = (out - predicate_matrix[labels] + diff_square).sum() / batch_size
    false_positives *= loss_cl.item() / (false_positives.item() + eps)

    missing_attr = (predicate_matrix[labels] - out + diff_square).sum() / batch_size
    missing_attr *= loss_cl.item() / (missing_attr.item() + eps)
    
    return loss_cl + false_positives/(missing_attr+eps) * FT_WEIGHT

def loss_fn(out, labels, predicate_matrix):
    out = out.view(-1, 1, NUM_FEATURES) # out is a batch of 1D binary vectors
    ANDed = out * predicate_matrix # AND operation
    diff = ANDed - out # Difference of ANDed and out => if equal, then out is a subset of its class' predicates

    entr_loss = torch.nn.CrossEntropyLoss()
    loss_cl = entr_loss(diff.sum(dim=2), labels) # Is "out" a subset of its class' predicates?

    batch_size = out.shape[0]

    out = out.view(-1, NUM_FEATURES)
    diff_square = (out - predicate_matrix[labels]).pow(2)
    
    false_positives = (out - predicate_matrix[labels] + diff_square).sum() / batch_size
    missing_attr = (predicate_matrix[labels] - out + diff_square).sum() / batch_size
    
    loss_ft = false_positives + missing_attr
    loss_ft *= loss_cl.item()/(loss_ft.item() + eps)
    
    return loss_cl + loss_ft * FT_WEIGHT

In [9]:
def train_one_epoch():
    running_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data["images"], data["labels"]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs, commit_loss, predicate_matrix = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels, predicate_matrix) + commit_loss
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        
        scheduler.step()

        # Gather data and report
        running_loss += loss.item()

    return running_loss / (i+1)

In [11]:
from torchmetrics import Accuracy
import sys, os
sys.path.insert(0, "/".join(os.path.abspath('').split("/")[:-1]) + "/models")
print("/".join(os.path.abspath('').split("/")[:-1]) + "/models")
from ResnetAutoPredicates import ResExtr

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

NUM_FEATURES = 48
NUM_CLASSES = 200
EPOCHS = 50
accuracy = Accuracy(task="multiclass", num_classes=NUM_CLASSES, top_k=1).to(device)

FT_WEIGHT = 0.5

model = ResExtr(NUM_FEATURES, NUM_CLASSES, pretrained=True).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 5e-4, epochs=EPOCHS, steps_per_epoch=len(training_loader))

best_stats = {
    "epoch": 0,
    "train_loss": 0,
    "val_loss": 0,
    "val_acc": 0,
    "fp": 0,
    "ma": 0,
    "oa": 0
}

from tqdm import tqdm
for epoch in tqdm(range(EPOCHS)):
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch()

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()
    running_acc = 0.0
    running_false_positives = 0.0
    running_missing_attr = 0.0
    running_out_attributes= 0.0

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata["images"], vdata["labels"]
            vinputs = vinputs.to(device)
            vlabels = vlabels.to(device)
            voutputs, vcommit_loss, predicate_matrix = model(vinputs)
            vloss = loss_fn(voutputs, vlabels, predicate_matrix) + vcommit_loss
            running_vloss += vloss.item()
            voutputs = voutputs.view(-1, 1, NUM_FEATURES)
            ANDed = voutputs * predicate_matrix
            diff = ANDed - voutputs
            running_acc += accuracy(diff.sum(dim=2), vlabels)
            voutputs = voutputs.view(-1, NUM_FEATURES)
            running_false_positives += ((predicate_matrix[vlabels] - voutputs) == -1).sum() / voutputs.shape[0]
            running_missing_attr += ((voutputs - predicate_matrix[vlabels]) == -1).sum() / voutputs.shape[0]
            running_out_attributes += voutputs.sum() / voutputs.shape[0]

    avg_vloss = running_vloss / (i + 1)
    avg_acc = running_acc / (i + 1)
    avg_fp = running_false_positives / (i + 1)
    avg_ma = running_missing_attr / (i + 1)
    avg_oa = running_out_attributes / (i + 1)
    print(f"LOSS: {avg_vloss}, ACC: {avg_acc}, FP: {avg_fp}, MA: {avg_ma}, OA: {avg_oa}")
    #print(model.bin_quantize._codebook.embed)
    
    with open("TINRes18AutoPredData.csv", "a") as f:
        f.write(f"{epoch}, {avg_loss}, {avg_vloss}, {avg_acc}, {avg_fp}, {avg_ma}, {avg_oa}\n")

    if best_stats["val_acc"] < avg_acc:
        best_stats["epoch"] = epoch
        best_stats["train_loss"] = avg_loss
        best_stats["val_loss"] = avg_vloss
        best_stats["val_acc"] = avg_acc.item()
        best_stats["fp"] = avg_fp.item()
        best_stats["ma"] = avg_ma.item()
        best_stats["oa"] = avg_oa.item()


print(best_stats)

/notebooks/Concept_ZSL/src/models
Device: cuda


  2%|▏         | 1/50 [00:15<12:31, 15.33s/it]

LOSS: 8.444142484664917, ACC: 0.02158203162252903, FP: 4.129101753234863, MA: 12.23496150970459, OA: 6.562597751617432


  4%|▍         | 2/50 [00:31<12:30, 15.63s/it]

LOSS: 8.190205073356628, ACC: 0.02685546875, FP: 3.4750001430511475, MA: 12.28623104095459, OA: 5.738574504852295


...

LOSS: 4.295258915424347, ACC: 0.49287110567092896, FP: 4.099218845367432, MA: 3.4544923305511475, OA: 11.477734565734863


 98%|█████████▊| 49/50 [12:57<00:15, 15.94s/it]

LOSS: 4.274431455135345, ACC: 0.49462890625, FP: 4.175879001617432, MA: 3.3988282680511475, OA: 11.605175971984863


100%|██████████| 50/50 [13:13<00:00, 15.88s/it]

LOSS: 4.279475277662277, ACC: 0.4950195252895355, FP: 4.172265529632568, MA: 3.40625, OA: 11.603906631469727
{'epoch': 35, 'train_loss': 1.744486412428834, 'val_loss': 4.246704167127609, 'val_acc': 0.49853515625, 'fp': 4.153613567352295, 'ma': 3.381542921066284, 'oa': 11.57431697845459}


# Resnet Baseline

In [7]:
def train_one_epoch_baseline():
    running_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data["images"], data["labels"]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn_baseline(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()

    return running_loss / (i+1)

In [13]:
from torchmetrics import Accuracy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

NUM_CLASSES = 200
EPOCHS = 15
accuracy = Accuracy(task="multiclass", num_classes=NUM_CLASSES, top_k=1).to(device)

POS_FT_WEIGHT = 0
FT_WEIGHT = 0

from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(512, NUM_CLASSES)

model = model.to(device)

loss_fn_baseline = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, weight_decay=1e-5)

best_stats = {
    "epoch": 0,
    "train_loss": 0,
    "val_loss": 0,
    "val_acc": 0,
}

from tqdm import tqdm
for epoch in tqdm(range(EPOCHS)):
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch_baseline()

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()
    running_acc = 0.0
    running_false_positives = 0.0

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata["images"], vdata["labels"]
            vinputs = vinputs.to(device)
            vlabels = vlabels.to(device)
            voutputs = model(vinputs)
            vloss = loss_fn_baseline(voutputs, vlabels)
            running_vloss += vloss.item()
            running_acc += accuracy(voutputs, vlabels)

    avg_vloss = running_vloss / (i + 1)
    avg_acc = running_acc / (i + 1)
    print(f"LOSS: {avg_vloss}, ACC: {avg_acc}")

    if best_stats["val_acc"] < avg_acc:
        best_stats["epoch"] = epoch
        best_stats["train_loss"] = avg_loss
        best_stats["val_loss"] = avg_vloss
        best_stats["val_acc"] = avg_acc.item()

print(best_stats)

Device: cuda


  7%|▋         | 1/15 [00:15<03:32, 15.20s/it]

LOSS: 2.7538333177566527, ACC: 0.3666932284832001


 13%|█▎        | 2/15 [00:30<03:16, 15.14s/it]

LOSS: 2.268373465538025, ACC: 0.45415934920310974


...

LOSS: 2.1348794221878054, ACC: 0.5197624564170837


 93%|█████████▎| 14/15 [03:33<00:15, 15.18s/it]

LOSS: 2.1823304891586304, ACC: 0.523909866809845


100%|██████████| 15/15 [03:48<00:00, 15.26s/it]

LOSS: 2.1891473531723022, ACC: 0.528218686580658
{'epoch': 7, 'train_loss': 1.1221286131411183, 'val_loss': 1.9414733171463012, 'val_acc': 0.5363759398460388}


# Optuna

In [3]:
import torch.nn as nn

eps = 1e-10
def loss_fn_optuna(out, labels, predicate_matrix, NUM_FEATURES, FT_WEIGHT):
    out = out.view(-1, 1, NUM_FEATURES) # out is a batch of 1D binary vectors
    ANDed = out * predicate_matrix # AND operation
    diff = ANDed - out # Difference of ANDed and out => if equal, then out is a subset of its class' predicates

    entr_loss = nn.CrossEntropyLoss()
    loss_cl = entr_loss(diff.sum(dim=2), labels) # Is "out" a subset of its class' predicates?

    batch_size = out.shape[0]

    out = out.view(-1, NUM_FEATURES)
    diff_square = (out - predicate_matrix[labels]).pow(2)
    
    false_positives = (out - predicate_matrix[labels] + diff_square).sum() / batch_size
    missing_attr = (predicate_matrix[labels] - out + diff_square).sum() / batch_size
    
    loss_ft = (1 + false_positives + missing_attr)
    loss_ft *= loss_cl.item()/(loss_ft.item() + eps)
    
    return loss_cl + loss_ft * FT_WEIGHT

from torchmetrics import Accuracy
def train_one_epoch_optuna(model, optimizer, scheduler, NUM_FEATURES, FT_WEIGHT):
    running_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data["images"], data["labels"]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs, commit_loss, predicate_matrix = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn_optuna(outputs, labels, predicate_matrix, NUM_FEATURES, FT_WEIGHT) + commit_loss
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        
        scheduler.step()

        # Gather data and report
        running_loss += loss.item()

    return running_loss / (i+1)

from tqdm import tqdm
from torch import optim

import sys, os
sys.path.insert(0, "/".join(os.path.abspath('').split("/")[:-1]) + "/models")
print("/".join(os.path.abspath('').split("/")[:-1]) + "/models")
from ResnetAutoPredicates import ResExtr

def objective(trial):
    global trial_num
    trial_num += 1
    print(f"Starting trial {trial_num}")
    NUM_FEATURES = trial.suggest_int("num_features", 4, 16)
    FT_WEIGHT = trial.suggest_float("ft_weight", 0, 1)
    # Generate the model.
    model = ResExtr(NUM_FEATURES*8, NUM_CLASSES, pretrained=True).to(device)

    EPOCHS = 50

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-4, 4e-3, log=True)
    
    if lr > max_lr:
        raise optuna.TrialPruned()
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=EPOCHS, steps_per_epoch=len(training_loader))

    best_acc = 0.0

    for epoch in tqdm(range(EPOCHS)):
        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        _ = train_one_epoch_optuna(model, optimizer, scheduler, NUM_FEATURES*8, FT_WEIGHT)

        model.eval()
        running_acc = 0.0

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, vdata in enumerate(validation_loader):
                vinputs, vlabels = vdata["images"], vdata["labels"]
                vinputs = vinputs.to(device)
                vlabels = vlabels.to(device)
                voutputs, _, predicate_matrix = model(vinputs)
                voutputs = voutputs.view(-1, 1, NUM_FEATURES*8)
                ANDed = voutputs * predicate_matrix
                diff = ANDed - voutputs
                running_acc += accuracy(diff.sum(dim=2), vlabels)

        avg_acc = running_acc / (i + 1)

        if avg_acc > best_acc:
            best_acc = avg_acc

        if epoch == 30 and best_acc < 0.3:
            raise optuna.TrialPruned()
        elif epoch == 15 and best_acc < 0.1:
            raise optuna.TrialPruned()
    
    return best_acc

/notebooks/Concept_ZSL/src/models


In [ ]:
import optuna

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

NUM_CLASSES = 200
accuracy = Accuracy(task="multiclass", num_classes=NUM_CLASSES, top_k=1).to(device)

trial_num = -1

study = optuna.create_study(direction="maximize", study_name='TIN-ResNet18-AutoPred', load_if_exists=True, storage='sqlite:///Optuna.db')
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    if key == "num_features":
        print("    {}: {}".format(key, value*8))
    else:
        print("    {}: {}".format(key, value))

Device: cuda


[I 2023-10-09 10:03:35,516] Using an existing study with name 'TIN-ResNet18-AutoPred' instead of creating a new one.


Starting trial 0


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 255MB/s]
100%|██████████| 50/50 [13:15<00:00, 15.92s/it]
[I 2023-10-09 10:16:52,492] Trial 134 finished with value: 0.508007824420929 and parameters: {'num_features': 11, 'ft_weight': 0.52094269759797, 'lr': 1.0017170923451517e-05, 'max_lr': 0.0004997788710074081}. Best is trial 32 with value: 0.5235351920127869.


Starting trial 1


...

100%|██████████| 50/50 [13:06<00:00, 15.74s/it]
[I 2023-10-09 11:08:54,366] Trial 138 finished with value: 0.5029296875 and parameters: {'num_features': 8, 'ft_weight': 0.5268294439754077, 'lr': 1.4954940285625902e-05, 'max_lr': 0.00035089089513476804}. Best is trial 32 with value: 0.5235351920127869.


Starting trial 5


100%|██████████| 50/50 [13:48<00:00, 16.58s/it]
[I 2023-10-09 11:22:43,747] Trial 139 finished with value: 0.5088867545127869 and parameters: {'num_features': 10, 'ft_weight': 0.496299707598186, 'lr': 1.2616341489098811e-05, 'max_lr': 0.0004067613027165046}. Best is trial 32 with value: 0.5235351920127869.


Starting trial 6


 20%|██        | 10/50 [02:46<11:02, 16.57s/it]